# Python script to plot data from 14CARR

In [ ]:
import matplotlib
import numpy as np
from netCDF4 import Dataset
from __future__ import print_function
import ipywidgets as widget
from IPython.display import display
from matplotlib import animation, rc
from IPython.display import HTML
from lxml import etree
import requests

In [ ]:
%matplotlib notebook

In [ ]:
def animate(i):
    print('Loading frame ' + str(i + 1) + '/' + str(frames))
    if variable_glob == 'norm':
        norm = np.sqrt(rootgrp[i].variables['U'][:, :]**2 +
                       rootgrp[i].variables['V'][:, :]**2)
        mycontour = matplotlib.pyplot.contourf(norm, 50, cmap=contourcolor)
    else:
        varplot = rootgrp[i].variables[variable_glob][:, :]
        mycontour = matplotlib.pyplot.contourf(varplot, 50, cmap=contourcolor)
    if vector_glob:
        matplotlib.pyplot.quiver(x, y, rootgrp[i].variables['u'][:, :],
                   rootgrp[i].variables['v'][:, :])
    return mycontour

In [ ]:
def g(folder, variable, vector, launch):
    url_base = 'http://servdap.legi.grenoble-inp.fr:80/opendap/coriolis/14CARR/ESTUARY3_160714/' + folder + '/'
    catalog_file = requests.get(url_base + 'Dalsa2.civ4.mproj/catalog.xml')
    tree = etree.fromstring(catalog_file.content)

    items = 0
    for children in tree[2]:
        if '.nc' in children.get("name"):
            items += 1

    if launch:
        file_name = []
        
        # GLobal variables
        global rootgrp
        global variable_glob
        global vector_glob
        global x
        global y
        global contourcolor
        global frames
        global ani
        
        vector_glob = vector
        variable_glob = variable
        rootgrp = []
        i = 0
        j = 100
        
        while j <= 130:
            print('Reading file ' + str(j) + '/' + str(items))
            file_name.append('/Dalsa2.civ4.mproj/img_' + str(int(j)) + '.nc')
            print(url_base + file_name[i])
            rootgrp.append(Dataset(url_base + file_name[i]))
            j += 1
            i += 1
        print('All files loaded')
        
        frames = i
        matplotlib.pyplot.clf()   
        fig_xsize = 15
        fig_ysize = 10
        font_size = 25
        markersize = 10
        fig = matplotlib.pyplot.figure(num=0, figsize=(fig_xsize, fig_ysize), dpi=60,
                                       facecolor='w', edgecolor='w')
        matplotlib.pyplot.rcParams.update({'font.size': font_size})
        matplotlib.pyplot.rcParams['lines.markersize'] = 10
        xaxis_name = 'i'
        yaxis_name = 'j'
        xlin = np.linspace(0, 0.1, 50)
        x, y = np.meshgrid(xlin, xlin)
        contourcolor = matplotlib.pyplot.cm.coolwarm
        mycontour = matplotlib.pyplot.contourf(np.empty([200, 200]), 50, cmap=contourcolor)
        ani = animation.FuncAnimation(fig, animate, frames=i, interval=1, blit=False)
        cbar = matplotlib.pyplot.colorbar(mycontour)
        cbar.ax.set_ylabel('Velocity (m/s)')
        matplotlib.pyplot.gca().set_aspect('equal')
        matplotlib.pyplot.xlabel(xaxis_name)
        matplotlib.pyplot.ylabel(yaxis_name)

In [ ]:
base_url = 'http://servdap.legi.grenoble-inp.fr:80/opendap/coriolis/14CARR/ESTUARY3_160714/'
catalog_file = requests.get(base_url + 'catalog.xml')
tree = etree.fromstring(catalog_file.content)
folders = []
for children in tree[2]:
    if not '.' in children.get('name'):
        folders.append(r'' + children.get('name'))

widget.interactive(g, 
                   folder=widget.Select(options=folders,
                                        value='ESTUARY3a',
                                        description='Folder : '
                                        ),
                   variable=widget.Select(options=['U', 'V', 'norm'],
                                          description='Select a variable to plot'
                                        ),
                   vector=widget.Checkbox(value=False,
                                          description='Display vectors'
                                         ),
                   launch=widget.ToggleButton(value=False,
                                              description= 'Load files'
                                             )
                   )

In [ ]:
HTML(ani.to_jshtml())